In [1]:
!pip install open3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408.6 MB 33 kB/s 
     |████████████████████████████████| 3.1 MB 28.1 MB/s 
     |████████████████████████████████| 8.8 MB 21.9 MB/s 
     |████████████████████████████████| 1.1 MB 52.4 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 346 kB 89.3 MB/s 
     |████████████████████████████████| 9.8 MB 50.6 MB/s 
     |████████████████████████████████| 423 kB 66.1 MB/s 
     |████████████████████████████████| 1.6 MB 47.5 MB/s 
     |████████████████████████████████| 1.2 MB 67.1 MB/s 
     |████████████████████████████████| 59 kB 7.4 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 80 kB 8.5 MB/s 
     |████████████████████████████████| 271 kB 36.5 MB/s 
     |████████████████████████████████| 133 kB 64.4 MB/s 
     |████████████████████████████████|

In [2]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Surface Reconstruction Tutorial

In [3]:
def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()
        
        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((1.0, 0.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            triangles = np.asarray(geometry.triangles)
            vertices = np.asarray(geometry.vertices)
            colors = None
            if geometry.has_vertex_colors():
                print('geometry.has_vertex_colors()')
                colors=np.asarray(geometry.vertex_colors)
            if geometry.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)
            print(colors)
            mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
            graph_objects.append(mesh_3d)
        
    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()
o3d.visualization.draw_geometries = draw_geometries

In [4]:
bunny = o3d.data.BunnyMesh()
mesh = o3d.io.read_triangle_mesh(bunny.path)
mesh.compute_vertex_normals()

pcd = mesh.sample_points_poisson_disk(750)
o3d.visualization.draw_geometries([pcd])
alpha = 0.02
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/BunnyMesh.ply
[Open3D INFO] Downloaded to /root/open3d_data/download/BunnyMesh/BunnyMesh.ply


alpha=0.020
((0.7792228973874724, 0.9010160207079972, 0.6059279222399278), (0.9677423744250184, 0.6600556481618708, 0.5748282076407638), (0.24485125546363296, 0.09313720544212017, 0.360853370266944), (0.7596710238040368, 0.9037375332509824, 0.6398819632439267), (0.48940752840269436, 0.0215900507638096, 0.3550458002771704), (0.9833171888991961, 0.48628627328091556, 0.3726562580491169), (0.03716471292019691, 0.5293273284171489, 0.6868780480464973), (0.16025689512960573, 0.1697835079977688, 0.34021360789239674), (0.5085270523367617, 0.05142983649352545, 0.27929182663475993), (0.8136395075832222, 0.8286382954056517, 0.7088710848246391), (0.13416672192228934, 0.33806589519482166, 0.2000944176047036), (0.4842410458532973, 0.07554439279555691, 0.23620255331616724), (0.8449434055050766, 0.24303579386643082, 0.7549185041630069), (0.6978884044368927, 0.04239577632304897, 0.5379282726002093), (0.27289079145955547, 0.9355461388010147, 0.5933861251560433), (0.09661194479591673, 0.7187207567335232, 

In [5]:
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([pcd, rec_mesh])

((0.020731605695971878, 0.6275668348314232, 0.4365294645340766), (0.9993596231334942, 0.4770651195231984, 0.5106751131839173), (0.996221833446523, 0.4749236072697085, 0.444008335097022), (0.9693775553085131, 0.3747308742238517, 0.3817107075856686), (0.9634706532497289, 0.3490246578096128, 0.38863842841103124), (0.9684982517262803, 0.327834351546559, 0.5294682476917948), (0.9404818647639099, 0.7010455535487635, 0.37527465288731465), (0.9774126058529231, 0.6231019664803131, 0.5832052259260279), (0.9730604310325361, 0.3686459303696956, 0.4053430563409724), (0.9335767724295351, 0.4072919299673593, 0.268880125994301), (0.9322835979574371, 0.4921118103652944, 0.24886524055547432), (0.9993626970847138, 0.5015702928561601, 0.5251879125896968), (0.86284435586754, 0.19400347069884777, 0.6571944575063609), (0.8138134776293993, 0.19144233299046215, 0.737304166393626), (0.7715430586818478, 0.11508029553404814, 0.6676340907916731), (0.6392673960372847, 0.01985698749269771, 0.5082024350900631), (0.61

In [6]:
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh])

[Open3D DEBUG] Input Points / Samples: 750 / 750
[Open3D DEBUG] #   Got kernel density: 0.0160670 (s), 281.371 (MB) / 281.371 (MB) / 281 (MB)
[Open3D DEBUG] #     Got normal field: 0.0131791 (s), 283.797 (MB) / 283.797 (MB) / 283 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.810449e-03 / 2.857836e+00
[Open3D DEBUG] #       Finalized tree: 0.0229011 (s), 289.211 (MB) / 289.211 (MB) / 289 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.108736 (s), 289.211 (MB) / 289.211 (MB) / 289 (MB)
[Open3D DEBUG] #Set point constraints: 0.0102170 (s), 289.211 (MB) / 289.211 (MB) / 289 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 138188 / 66888 / 91041
[Open3D DEBUG] Memory Usage: 289.211 MB
[Open3D DEBUG] # Linear system solved: 0.153658 (s), 289.211 (MB) / 289.211 (MB) / 289 (MB)
[Open3D DEBUG] Got average: 0.00648308 (s), 289.211 (MB) / 289.211 (MB) / 289 (MB)
[Open3D DEBUG] Iso-Value: 5.025454e-01 = 3.769090e+02 / 7.500000e+02
[Open3D DEBUG] #          Total Solve:       1.1 (

# Surface Reconstruction from ARCore's point clouds

In [7]:
from sklearn.preprocessing import MinMaxScaler
def reshapePointCloudData(pointCloud):
  column=6
  row=int(pointCloud.shape[0]/column)
  pointCloud=np.reshape(pointCloud,(row,column))
  pointCloud=pointCloud[~np.all(pointCloud == 0, axis=1)]
  pointCloud=np.hsplit(pointCloud, 2)
  scaler = MinMaxScaler()
  pointCloud[1]=scaler.fit_transform(pointCloud[1])
  return pointCloud[0],pointCloud[1]
def constructPointCloud(numOfImage):
  rawDepthPointCloudData=[]
  for i in range(numOfImage):
    tempData= np.fromfile("rawDepthPointCloudData ("+str(i)+").txt", dtype = np.float32)
    tempData=reshapePointCloudData(tempData)
    print('tempData.shape ',tempData.shape)
    if(rawDepthPointCloudData==[]):
      rawDepthPointCloudData=tempData
    else:
      rawDepthPointCloudData=np.vstack((rawDepthPointCloudData,tempData))

  return rawDepthPointCloudData

In [8]:
rawDepthPointCloudData= np.fromfile("rawDepthPointCloudData.txt", dtype = np.float32)
pointcloud,rgb=reshapePointCloudData(rawDepthPointCloudData)
print(pointcloud)
print(rgb)

[[ 0.29846922 -0.837816   -1.6183815 ]
 [ 0.29966652 -0.84528947 -1.5985832 ]
 [ 0.3008945  -0.8532034  -1.5810442 ]
 ...
 [ 0.03431636 -0.83974445 -0.2563203 ]
 [ 0.06581539 -0.8214816  -0.25778678]
 [ 0.07461035 -0.82229155 -0.25614065]]
[[0.12019231 0.10191083 0.06164384]
 [0.13461539 0.12738854 0.09589041]
 [0.13461539 0.12101911 0.07534246]
 ...
 [0.5865385  0.7388535  0.7328767 ]
 [0.45673078 0.5923567  0.5479452 ]
 [0.6057693  0.7197453  0.71232873]]


In [9]:
pointcloud.shape

(21700, 3)

In [10]:
pointcloud

array([[ 0.29846922, -0.837816  , -1.6183815 ],
       [ 0.29966652, -0.84528947, -1.5985832 ],
       [ 0.3008945 , -0.8532034 , -1.5810442 ],
       ...,
       [ 0.03431636, -0.83974445, -0.2563203 ],
       [ 0.06581539, -0.8214816 , -0.25778678],
       [ 0.07461035, -0.82229155, -0.25614065]], dtype=float32)

In [11]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pointcloud)
pcd.colors = o3d.utility.Vector3dVector(rgb)
o3d.visualization.draw_geometries([pcd])

Output hidden; open in https://colab.research.google.com to view.

In [12]:
np.asarray(mesh.vertex_colors)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [13]:
alpha = 0.02
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

Output hidden; open in https://colab.research.google.com to view.

In [14]:
radii = [0.005, 0.01, 0.02, 0.04]
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([rec_mesh])

Output hidden; open in https://colab.research.google.com to view.

In [15]:
rgb

array([[0.12019231, 0.10191083, 0.06164384],
       [0.13461539, 0.12738854, 0.09589041],
       [0.13461539, 0.12101911, 0.07534246],
       ...,
       [0.5865385 , 0.7388535 , 0.7328767 ],
       [0.45673078, 0.5923567 , 0.5479452 ],
       [0.6057693 , 0.7197453 , 0.71232873]], dtype=float32)

In [16]:
rgb

array([[0.12019231, 0.10191083, 0.06164384],
       [0.13461539, 0.12738854, 0.09589041],
       [0.13461539, 0.12101911, 0.07534246],
       ...,
       [0.5865385 , 0.7388535 , 0.7328767 ],
       [0.45673078, 0.5923567 , 0.5479452 ],
       [0.6057693 , 0.7197453 , 0.71232873]], dtype=float32)

In [17]:
rec_mesh.vertex_colors=o3d.utility.Vector3dVector(rgb)

In [18]:
np.asarray(rec_mesh.vertex_colors)

array([[0.12019231, 0.10191083, 0.06164384],
       [0.13461539, 0.12738854, 0.09589041],
       [0.13461539, 0.12101911, 0.07534246],
       ...,
       [0.58653849, 0.73885351, 0.73287672],
       [0.45673078, 0.59235668, 0.5479452 ],
       [0.60576928, 0.71974528, 0.71232873]])

In [19]:
o3d.visualization.draw_geometries([rec_mesh])

Output hidden; open in https://colab.research.google.com to view.

In [20]:
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh])

Output hidden; open in https://colab.research.google.com to view.